# Creating a Sampled Dataset

**Learning Objectives**
- Sample the natality dataset to create train/eval/test sets
- Preprocess the data in Pandas dataframe

## Introduction

In this notebook we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe. 

In [1]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.14"                # TF version for CMLE to use

In [2]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets.

In [4]:
# Create SQL query using natality data after the year 2000
query_string = """
WITH
  CTE_hash_cols_fixed AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
      WHEN day IS NULL AND wday IS NULL THEN 0
    ELSE
    CASE
      WHEN day IS NULL THEN wday
    ELSE
    wday
  END
  END
    AS date,
    IFNULL(state,
      "Unknown") AS state,
    IFNULL(mother_birth_state,
      "Unknown") AS mother_birth_state
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000)

SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING), CAST(date AS STRING), CAST(state AS STRING), CAST(mother_birth_state AS STRING)))) AS hashvalues
FROM
  CTE_hash_cols_fixed
"""

There are only a limited number of years, months, days, and states in the dataset. Let's see what the hash values are.

We'll call BigQuery but group by the hashcolumn and see the number of records for each group. This will enable us to get the correct train/eval/test percentages

In [12]:
from google.cloud import bigquery
bq = bigquery.Client(project = PROJECT)

df = bq.query("SELECT count(*) as cnt FROM (" 
              + query_string + 
              ") ").to_dataframe()
df

,cnt
0,33271914


In [13]:
33271914/500

66543.828

In [5]:
from google.cloud import bigquery
bq = bigquery.Client(project = PROJECT)

df = bq.query("SELECT hashvalues, COUNT(weight_pounds) AS num_babies FROM (" 
              + query_string + 
              ") GROUP BY hashvalues").to_dataframe()

print("There are {} unique hashvalues.".format(len(df)))
df.head()

There are 658107 unique hashvalues.


,hashvalues,num_babies
0,2152741494508244169,226
1,2890037128296213246,79
2,1698051122368355036,2068
3,1446577703205389352,1321
4,6101796616645784663,2850


We can make a query to check if our bucketing values result in the correct sizes of each of our dataset splits and then adjust accordingly

In [6]:
sampling_percentages_query = """
WITH
  -- Get label, features, and column that we are going to use to split into buckets on
  CTE_hash_cols_fixed AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
      WHEN day IS NULL AND wday IS NULL THEN 0
    ELSE
    CASE
      WHEN day IS NULL THEN wday
    ELSE
    wday
  END
  END
    AS date,
    IFNULL(state,
      "Unknown") AS state,
    IFNULL(mother_birth_state,
      "Unknown") AS mother_birth_state
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000),
  CTE_data AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(year AS STRING), CAST(month AS STRING), CAST(date AS STRING), CAST(state AS STRING), CAST(mother_birth_state AS STRING)))) AS hashvalues
  FROM
    CTE_hash_cols_fixed),
  -- Get the counts of each of the unique hashs of our splitting column
  CTE_first_bucketing AS (
  SELECT
    hashvalues,
    COUNT(*) AS num_records
  FROM
    CTE_data
  GROUP BY
    hashvalues ),
  -- Get the number of records in each of the hash buckets
  CTE_second_bucketing AS (
  SELECT
    MOD(hashvalues, {0}) AS bucket_index,
    SUM(num_records) AS num_records
  FROM
    CTE_first_bucketing
  GROUP BY
    MOD(hashvalues, {0})),
  -- Calculate the overall percentages
  CTE_percentages AS (
  SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
      SUM(num_records)
    FROM
      CTE_second_bucketing) AS percent_records
  FROM
    CTE_second_bucketing ),
  -- Choose which of the hash buckets will be used for training and pull in their statistics
  CTE_train AS (
  SELECT
    *,
    "train" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= 0
    AND bucket_index < {1}),
  -- Choose which of the hash buckets will be used for validation and pull in their statistics
  CTE_eval AS (
  SELECT
    *,
    "eval" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {1}
    AND bucket_index < {2}),
  -- Choose which of the hash buckets will be used for testing and pull in their statistics
  CTE_test AS (
  SELECT
    *,
    "test" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {2}
    AND bucket_index < {0}),
  -- Union the training, validation, and testing dataset statistics
  CTE_union AS (
  SELECT
    0 AS dataset_id,
    *
  FROM
    CTE_train
  UNION ALL
  SELECT
    1 AS dataset_id,
    *
  FROM
    CTE_eval
  UNION ALL
  SELECT
    2 AS dataset_id,
    *
  FROM
    CTE_test ),
  -- Show final splitting and associated statistics
  CTE_split AS (
  SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
  FROM
    CTE_union
  GROUP BY
    dataset_id,
    dataset_name )
SELECT
  *
FROM
  CTE_split
ORDER BY
    dataset_id
"""

modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

df = bq.query(sampling_percentages_query.format(modulo_divisor, train_buckets, train_buckets + eval_buckets)).to_dataframe()
df.head()

,dataset_id,dataset_name,num_records,percent_records
0,0,train,26080035,0.783845
1,1,eval,3639721,0.109393
2,2,test,3552158,0.106761


#### **Exercise 1**

Modify the `query_string` above so to produce a 80/10/10 split for the train/valid/test set. Use the `hashvalues` taking an appropriate `MOD()` value.

**Hint**: You can use every_n in the SQL query to create a smaller subset of the data

In [14]:
# Added every_n so that we can now subsample from each of the hash values to get approximately the record counts we want
every_n = 500

train_query = 'Select * From ({0}) Where MOD(hashvalues, 10) < 8 And Rand() < 1/{1} '.format(query_string,every_n)
eval_query = 'Select * From ({0}) Where MOD(hashvalues, 10) = 8 And Rand() < 1/{1} '.format(query_string,every_n)
test_query = 'Select * From ({0}) Where MOD(hashvalues, 10) = 9 And Rand() < 1/{1} '.format(query_string,every_n)

train_df = bq.query(train_query).to_dataframe()
eval_df = bq.query(eval_query).to_dataframe()
test_df = bq.query(test_query).to_dataframe()

print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 52678 examples in the train dataset.
There are 6898 examples in the validation dataset.
There are 6897 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`. Also, if there is more than child we'll change the `plurality` to `Multiple(2+)`. While we're at it, We'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [9]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
0,8.437091,False,15,1,36.0,3277484076117500784
1,7.125340,False,49,1,42.0,1222148953150957267
2,3.249614,True,15,1,35.0,1661075941425199601
3,7.032746,True,15,1,34.0,605995451125822142
4,5.328573,True,50,2,39.0,6000209594394293837


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [10]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashvalues
count,52754.000000,52816.000000,52816.000000,52427.000000,5.281600e+04
mean,7.239603,27.365685,1.033967,38.629351,4.681894e+18
std,1.317665,6.167324,0.189623,2.572526,2.671965e+18
min,0.500449,13.000000,1.000000,17.000000,6.912390e+14
25%,6.563162,22.000000,1.000000,38.000000,2.426878e+18
50%,7.312733,27.000000,1.000000,39.000000,4.734591e+18
75%,8.062305,32.000000,1.000000,40.000000,6.978911e+18
max,13.503314,54.000000,5.000000,47.000000,9.222032e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

#### **Exercise 2**

The code cell below has some TODOs for you to complete.

In the first block of TODOs, we'll clean the data so that 
- `weight_pounds` is always positive
- `mother_age` is always positive
- `gestation_weeks` is always positive
- `plurality` is always positive

The next block of TODOs will create extra rows to simulate lack of ultrasound information. That is, we'll make a copy of the dataframe and call it `no_ultrasound`. Then, use Pandas functionality to make two changes in place to `no_ultrasound`:
- set the `plurality` value of `no_ultrasound` to be 'Multiple(2+)' whenever the plurality is not 'Single(1)'
- set the `is_male` value of `no_ultrasound` to be 'Unknown'

In [15]:
import pandas as pd

def preprocess(df):
    # Clean up data
    # Remove what we don"t want to use for training
    df = df[df['weight_pounds'] > 0]
    df = df[df['mother_age'] > 0]
    df = df[df['gestation_weeks'] > 0]
    df = df[df['plurality'] > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace=True)

    # Now create extra rows to simulate lack of ultrasound
    no_ultrasound = df.copy(deep=True)
    no_ultrasound.loc[no_ultrasound["plurality"] != "Single(1)", "plurality"] = "Multiple(2+)"
    no_ultrasound["is_male"] = "Unknown"
    
    # Concatenate both datasets together and shuffle
    return pd.concat([df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

Let's process the train/eval/test set and see a small sample of the training data after our preprocessing:

In [16]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [17]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
0,8.419454,False,21,Single(1),38.0,5856262074638829622
1,8.062305,True,27,Single(1),38.0,6038339242779302767
2,6.124442,Unknown,37,Single(1),38.0,5966841057706794787
3,9.376260,Unknown,25,Single(1),40.0,6330814442248885480
4,7.561856,False,35,Single(1),39.0,1545048565354170783


In [18]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashvalues
104561,6.437498,True,34,Single(1),35.0,4435318956967134360
104562,7.187070,True,35,Single(1),38.0,966542863346049620
104563,8.000575,Unknown,23,Single(1),39.0,2530015940142889432
104564,6.812284,False,25,Single(1),39.0,8554308952820143753
104565,7.687519,Unknown,31,Single(1),40.0,1382017907614137740


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [ ]:
train_df.describe()

## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

#### **Exercise 3**

Complete the code in the cell below to write the the three Pandas dataframes you made above to csv files. Have a look at [the documentation for `.to_csv`]( https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) to remind yourself its usage. Remove `hashvalues` from the data since we will not be using it in training so there is no need to move around extra data.

In [26]:
columns = ['weight_pounds','is_male','mother_age','plurality','gestation_weeks']
train_df.to_csv('train.csv', columns =columns, header = False, index = False)
eval_df.to_csv('eval.csv', columns =columns, header = False, index = False)
test_df.to_csv('test.csv', columns =columns, header = False, index = False)

Check your work above by inspecting the files you made. 

In [27]:
%%bash
wc -l *.csv

  13702 eval.csv
  13696 test.csv
 104566 train.csv
 131964 total


In [28]:
%%bash
head *.csv

==> eval.csv <==
5.87311465968,Unknown,29,Single(1),35.0
7.62578964258,True,38,Single(1),37.0
7.5067400211,Unknown,26,Single(1),41.0
7.5618555866,False,37,Single(1),38.0
8.811876612139999,Unknown,26,Single(1),39.0
8.313631900019999,True,21,Single(1),40.0
8.6641668966,Unknown,27,Single(1),40.0
7.50012615324,True,19,Single(1),42.0
7.62578964258,True,25,Single(1),38.0
6.92692427204,Unknown,27,Single(1),40.0

==> test.csv <==
4.68702769012,False,39,Single(1),34.0
5.96570880972,Unknown,36,Single(1),26.0
6.85858097082,True,21,Single(1),40.0
8.50102482272,False,43,Single(1),38.0
7.936641432,True,23,Single(1),38.0
10.1853565044,True,27,Single(1),40.0
6.3382900325,Unknown,23,Single(1),42.0
8.18796841068,Unknown,18,Single(1),40.0
8.93754010148,True,34,Single(1),38.0
6.52347833258,False,27,Single(1),39.0

==> train.csv <==
8.41945378578,False,21,Single(1),38.0
8.062304921339999,True,27,Single(1),38.0
6.1244416383599996,Unknown,37,Single(1),38.0
9.37626000286,Unknown,25,Single(1),40.0
7.5618555866

In [22]:
%%bash
tail *.csv

==> eval.csv <==
13692,6.60064012428,Unknown,30,Single(1),40.0,6222150371233341678
13693,8.93754010148,True,26,Single(1),39.0,7428257040318018688
13694,9.8767093376,False,19,Single(1),40.0,3052313262559742378
13695,7.06361087448,False,31,Single(1),38.0,6828080505975406908
13696,5.43659938092,Unknown,36,Single(1),34.0,5295336292124293138
13697,7.5618555866,True,34,Single(1),38.0,5158335928726137738
13698,8.000575487979999,False,23,Single(1),39.0,7164177241529478448
13699,6.2501051276999995,Unknown,22,Single(1),40.0,4923281938564606528
13700,3.18788430852,False,21,Single(1),28.0,3620580808694281848
13701,6.5477291814,Unknown,24,Single(1),38.0,3844555583997440408

==> test.csv <==
13686,8.12623897732,Unknown,17,Single(1),38.0,5452070256122607779
13687,6.0627122049999995,Unknown,29,Single(1),38.0,1882494054335134209
13688,6.0627122049999995,Unknown,24,Single(1),43.0,4104007969205529769
13689,7.43839671988,Unknown,25,Single(1),38.0,7308799370622112259
13690,6.37576861704,False,23,Single(1),

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License